In [31]:
import os
import sys
import logging
import json
sys.path.append(os.path.join('/gpfs/home/bsk18/factual-bias-mitigation/src/contrastive_learning/', '../../'))

from src.contrastive_learning.utils import (
    print_gpu_memory, 
    read_files, 
    combine_data,
    calculate_original_factuality,
    calculate_original_toxicity,
    generate_dataset_name,
)
from src.contrastive_learning.train_utils import(
    combined_loss,
    evaluate_w_toxicity,
)
from src.contrastive_learning.model import SummarizationModel
from src.contrastive_learning.dataset import SummarizationDataset 

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW 
from tqdm import tqdm
from transformers import AutoModel, GPT2Tokenizer, AutoModelForCausalLM
import random
random.seed(42)
import wandb
import argparse
from datetime import datetime

# Set up logging
import logging
import os
from datetime import datetime


In [32]:
# Process 'all' option
args = {'pos_data': 'all'}
all_pos_options = ['original', 'bt']    
if 'all' in args["pos_data"]:
    args["pos_data"] = all_pos_options

# Ensure 'original' is always included
if 'original' not in args["pos_data"]:
    args["pos_data"].insert(0, 'original')

# Define data paths based on arguments
pos_data_paths = ['/gpfs/home/bsk18/factual-bias-mitigation/data/tldr/pos']

for opt in args["pos_data"]:
    if opt != 'original':
        pos_data_paths.append(f'/gpfs/home/bsk18/factual-bias-mitigation/data/tldr/pos_{opt}')

# Combine data from all specified paths
logger = logging.getLogger()
pos_data = combine_data([read_files(path, logger) for path in pos_data_paths])

# Initialize the model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained('gpt2')

# Set pad_token_id in model configuration
model.config.pad_token_id = tokenizer.eos_token_id


# Initialize datasets
train_dataset = SummarizationDataset(pos_data['train'], tokenizer, max_length=512)
val_dataset = SummarizationDataset(pos_data['validation'], tokenizer, max_length=512)
test_dataset = SummarizationDataset(pos_data['test'], tokenizer, max_length=512)

# Create config dictionary
config = {
    "ce_weight": 1,
    "model_name": model_name,
    "batch_size": 4,
    "learning_rate": 1e-4,
    "epochs": 10,
}

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"])
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"])
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"])


In [33]:
it = iter(train_loader)
data = next(it)
data

{'input_ids': tensor([[50256, 50256, 50256,  ...,   783,    13, 50256],
         [50256, 50256, 50256,  ...,  8855,    13, 50256],
         [50256, 50256, 50256,  ..., 48389,    13, 50256],
         [13065,  3876,  1096,  ...,  1182,    11,   314]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([[ -100,  -100,  -100,  ...,   783,    13, 50256],
         [ -100,  -100,  -100,  ...,  8855,    13, 50256],
         [ -100,  -100,  -100,  ..., 48389,    13, 50256],
         [ -100,  -100,  -100,  ...,  -100,  -100,  -100]]),
 'source_text': ["Summarize: I'll chime in before this gets buried. Although I'm only 22 myself (bearing in mind the drinking age is 18 here), it's certainly true that even since I've been drinking there has been massive growth in interest in craft beer and 'real ale' as it'll often be called here (although arguably different things... For

In [34]:
pos_data['train']['source'][1]

'Summarize: A friend left my house with another friend. He left his car in my driveway. A couple of other buddies were at my house and suggested we take his car and go jump it over the railroad track. There was a dirt road that had a railroad track at the top of a hill. There was a rapid drop off on the other side, so you could get considerable air. "Ok," I say and we head out. We jump it a few times before the car\'s transmission goes out. The car would only go in reverse. We back the car back home in reverse. We were miles from my house and we even had to back it down main street. We put it in the exact spot we found it. The friend comes back, says his good-byes and gets in his car to go home. Shortly, he comes in the house furious. "The jerk sold me a lemon! I\'ve only had the car a week and now it\'s leaking transmission fluid!" It was extremely hard for us to keep a straight face. Before anyone calls us jerks, the car wasn\'t worth much and we got it all fixed up. I told him about

In [23]:
print("Source Input IDs:", source_encoding['input_ids'])
print("Target Input IDs:", target_encoding['input_ids'])

# Decode back to text
decoded_source = tokenizer.decode(source_encoding['input_ids'][0], skip_special_tokens=True)
decoded_target = tokenizer.decode(target_encoding['input_ids'][0], skip_special_tokens=True)

print("Decoded Source:", decoded_source)
print("Decoded Target:", decoded_target)

NameError: name 'source_encoding' is not defined